In [3]:
from IPython.display import display, Math
from sympy.physics.quantum import TensorProduct
from symboliq.dirac_notation import DiracNotation, cx, h, i, ket_0, x, ket_1, b_0, B0

In [6]:
dirac_notation = DiracNotation(B0 * ket_0)

In [10]:
res = dirac_notation.get_steps_latex()

In [11]:
display(Math(res))

<IPython.core.display.Math object>